# library loading

In [1]:
# 필수 라이브러리 로딩
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver import ActionChains # 스크롤
from urllib import request # 이미지
import csv
import time
# page로딩과 구조 로딩에 gab이 있어서 원하는 tag나올떄까지 기다리는거
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import numpy as np # 결측값 삽입용
import re # 정규표현식
import pandas as pd

# 변수선언

In [2]:
### new
ref_info = [] # 소개 출처 수집용
ref_review = [] # 서평 출처 수집용
ref_index = [] # 목차 출처 수집용
## 교재 테이블 리스트들 
main_title = []  # 본제
sub_title = []  # 부제
author = []  # 저자
publish = []  # 출판
date = []  # 발행일
cate = []  # 네이버 카테고리
cate_kb = [] # 교보 카테고리
tot_page = []  # 전체 쪽수
size = []  # 크기
wei = []  # 무게
img = [] # 이미지
book_info = []  # 책소개
book_review = []  # 책서평
kobo_cate = []
title_isbn = [] # isbn 교재명
set_isbn = [] #set
isbn = []
book_type = [] #제본
ver = []  # 판정보
last_ver = []  # 이전 판 정보
pn_isbn = [] # 페이지 관련 끌기
keyword = []  # 키워드
price = []

index = []
idx_page = []

# 중복 제목시 제거를 위한 리스트
over_ls = []

# 관련 사전 정의
info_dic = {'author':author, 'publish': publish, 'date':date, '카테고리':cate, 'isbn':isbn}
isbn_dic = {'isbn':isbn, 'book_type': book_type, 'ver':ver,'last_ver':last_ver ,'pn_isbn':pn_isbn ,\
           'keyword':keyword, 'set_isbn': set_isbn, 'price':price}
plus_dic = {'book_info':book_info, 'book_review': book_review, 'index':index}
plus_ref_dic = {'book_info':ref_info, 'book_review': ref_review, 'index':ref_index}

In [3]:
def info_func():
    info = driver.find_elements(By.CLASS_NAME, "bookTitle_item_info__IMLyY")
    check = []
    for i in range(len(info)):
        a = info[i].text.split('\n')
        if a[0] == '저자':
            check.append('author')
            aut = a[1] # 저자
        elif a[0] == '출판':
            check.append('publish')
            pub = a[1]
        elif a[0] == '발행':
            check.append('date')
            
            if a[1].split('.')[0] != 2023:
                dat ='X'
                print('exit')
            else:
                dat = a[1]
        else:
            pass
    return aut, pub, dat

In [4]:
# 저자, 출판, 발행일 수집 함수
'''def info_func():
    info = driver.find_elements(By.CLASS_NAME, "bookTitle_item_info__IMLyY")
    check = []
    for i in range(len(info)):
        a = info[i].text.split('\n')
        if a[0] == '저자':
            check.append('author')
            author.append(a[1]) # 저자
        elif a[0] == '출판':
            check.append('publish')
            publish.append(a[1])
        elif a[0] == '발행':
            check.append('date')
            date.append(a[1])
        else:
            pass
    for c in ['author','publish', 'date']:
        if c not in check:
            info_dic[c].append(np.NaN)'''

# 네이버 카테고리, isbn번호 수집  
def sub1_func():
    cate_ls = driver.find_elements(By.CLASS_NAME, "bookBasicInfo_inner__YIfRy")
    check = []
    for i in range(len(cate_ls)):
        a = cate_ls[i].text.split('\n')
        if '카' in a[0]:
            check.append('카테고리')
            cate.append(a[1]) # 카테고리
        elif a[0] == 'ISBN':
            check.append('isbn')
            code = a[1]
        else:
            pass
        if '카테고리' not in check:
            cate.append(np.NaN)
        elif 'isbn' not in check:
            code = 'O' #기존 수집되는 isbn도 소수형으로 수집되어서 문자로 대체
    return code

# 페이지수, 무게, 크기 수집
def sub2_func():
    sub2 = driver.find_elements(By.CLASS_NAME, "bookBasicInfo_spec__qmQ_N") # 쪽수, 크기(무게)
    if len(sub2) == 3:
        tot_page.append(sub2[0].text)
        # wei
        wei.append(sub2[1].text)
        # size
        size.append(sub2[2].text)
    elif len(sub2) == 2:
        tot_page.append(sub2[0].text)
        # wei
        wei.append(np.NaN)
        # size
        size.append(sub2[1].text)
    elif len(sub2) == 1:
        tot_page.append(sub2[0].text)
        wei.append(np.NaN)
        size.append(np.NaN)
    else:
        tot_page.append(np.NaN)
        wei.append(np.NaN)
        size.append(np.NaN)

# 이미지 수집
def img_func(code):
    # image
    img_url = driver.find_element(By.XPATH, '//*[@id="container"]/div[2]/div[1]/div[1]/div[1]/div[1]/div/img').get_attribute("src")
    driver.implicitly_wait(10)
    request.urlretrieve(img_url,f"C:\\police_data\\police_img\\{code}.jpg")
    img.append(f'{code}.jpg')

# 목차 수집시 행별 페이지수 수집
def pages(a_text):
    t_ls = []
    for x in a_text[1:-1]:
        if x.split(' ')[-1].isnumeric() == True: # 수정함
            t_ls.append(x.split(' ')[-1]) # sub_p
        else:
            t_ls.append(np.NaN)
    return t_ls


In [5]:
def new_idx():
    plus_info = driver.find_elements(By.CLASS_NAME,'infoItem_info_item__1Nm8P')
    plus_ls = []
    ref = []
    for a in plus_info:
        a_text = a.text.split('\n')
        if a_text[0]== '책 소개':
            a_text = [t for t in a_text if t != '' ]
            book_info.append(a_text[1:-1])
            ref_info.append(f'소개 {a_text[-1][:4]}')
            plus_ls.append('book_info')
            
        elif a_text[0]== '출판사 서평':
            a_text = [t for t in a_text if t != '' ]
            book_review.append(a_text[1:-1])
            ref_review.append(f'서평 {a_text[-1][:4]}')
            plus_ls.append('book_review')
            
        elif a_text[0]== '목차':
            a_text = [t for t in a_text if t != '' ]
            index.append(a_text[1:-1])
            plus_ls.append('index')
            ref_index.append(f'목차 {a_text[-1][:4]}')
            t_ls = pages(a_text)
            idx_page.append(t_ls)
        else:
            pass
   # ref_ls.append(ref) # 신규
    return plus_ls

In [6]:
# 교보문고 category 수집
def kobo():
    driver.find_element(By.PARTIAL_LINK_TEXT, '교보').click()
    driver.switch_to.window(driver.window_handles[2]) 
    driver.implicitly_wait(10)
    try:
        kb = driver.find_element(By.CLASS_NAME, 'category_list_item').text
        kb = kb.split('>')
        kb = [k.strip() for k in kb]
        cate_kb.append(list(set(kb)))
    except:
        cate_kb.append(np.NaN)
    driver.close()
    driver.implicitly_wait(10)
    driver.switch_to.window(driver.window_handles[1])
    driver.implicitly_wait(10)
    print('정보수집kb')

In [7]:
# 네이보 도서내에서 실행될 함수
def naver():
    # 창 전환
    driver.switch_to.window(driver.window_handles[1]) 
    driver.implicitly_wait(10)
    try: # 제목 수집 -부제가 없는지 확인먼저 
        s_tit = driver.find_element(By.CLASS_NAME,f'bookTitle_sub_title__B0uMS').text
        m_tit = driver.find_element(By.CLASS_NAME,f'bookTitle_book_name__JuBQ2').text
        if m_tit not in over_ls:
            check = 0 # 계속 진행
        else:
            check = 1 # 나감
            
    except:
        m_tit = driver.find_element(By.CLASS_NAME,f'bookTitle_title_area__fspvB').text
        s_tit = np.NaN
        if m_tit not in over_ls:
            check = 0
        else:
            check = 1
    try:
        aut, pub, dat = info_func() # 저자, 출판, 발행일 수집
        if dat == 'X':
            check += 1
            # 수집 없이 지나칠거임(check가 1 또는 2)
        else:
            pass
    except:
        aut, pub, dat = np.NaN, np.NaN, np.NaN  # 해당 정보가 없으면 널 값

        # 수집할 데이터
    if check == 0:
        # 위에서 수집한, 제목 출판일등 저장
        over_ls.append(m_tit) # 중복제거를 위한
        main_title.append(m_tit) # 본
        sub_title.append(s_tit) # 부
        author.append(aut) # 저자
        publish.append(pub) # 출판
        date.append(dat) # 날짜
        try:
            code = sub1_func() # 카테고리, isbn 수집
        except:
            print('sub1함수 예외상황')

        try:
            sub2_func() # 페이지, 책 규격
        except:
            print('sub2함수 예외상황')

        try:
            img_func(code) # 이미지
        except:
            img.append(np.nan)
        # 교보문고 링크 존재여부 확인   
        book_text = driver.find_elements(By.CLASS_NAME, 'bookSeller_mall_name__S0crJ')
        book_text = [bt.text.split('\n')[0] for bt in book_text]
        if '교보문고' in book_text:
            kobo() # 교보문고 카테고리 수집
        else:
            cate_kb.append(np.NaN)    

        plus_ls = new_idx() # 소개, 서평, 목차 수집
        for p in ['book_info','book_review','index']:
            if p not in plus_ls:
                plus_dic[p].append(np.NaN) # 수집된 결과 없을시 널 값 부여
                plus_ref_dic[p].append(np.NaN) # 수집된 결과 없을시 널 값 부여
    else:
        code = 'X'
        
    return code

In [8]:
# 네이보 도서내에서 실행될 함수
'''def naver():
    # 창 전환
    driver.switch_to.window(driver.window_handles[1]) 
    driver.implicitly_wait(10)
    try: # 제목 수집 -부제가 없는지 확인먼저 
        s_tit = driver.find_element(By.CLASS_NAME,f'bookTitle_sub_title__B0uMS').text
        m_tit = driver.find_element(By.CLASS_NAME,f'bookTitle_book_name__JuBQ2').text
        if m_tit not in over_ls:
            over_ls.append(m_tit)
            main_title.append(m_tit) # 본
            sub_title.append(s_tit) # 부
            check = 0 # 계속 진행
        else:
            check = 1 # 나감
            
            
        
    except:
        m_tit = driver.find_element(By.CLASS_NAME,f'bookTitle_title_area__fspvB').text
        if m_tit not in over_ls:
            over_ls.append(m_tit)
            main_title.append(m_tit) # 본
            sub_title.append(np.NaN) # 부
            check = 0
        else:
            check = 1

    if check == 0:
        driver.implicitly_wait(10)
        try:
            info_func() # 저자, 출판, 발행일 수집
        except:
            for x in [author, publish, date]:
                x.append(np.NaN)  # 해당 정보가 없으면 널 값

        try:
            code = sub1_func() # 카테고리, isbn 수집
        except:
            print('sub1함수 예외상황')

        try:
            sub2_func() # 페이지, 책 규격
        except:
            print('sub2함수 예외상황')

        try:
            img_func(code) # 이미지
        except:
            img.append(np.nan)
        # 교보문고 링크 존재여부 확인   
        book_text = driver.find_elements(By.CLASS_NAME, 'bookSeller_mall_name__S0crJ')
        book_text = [bt.text.split('\n')[0] for bt in book_text]
        if '교보문고' in book_text:
            kobo() # 교보문고 카테고리 수집
        else:
            cate_kb.append(np.NaN)    

        plus_ls = new_idx() # 소개, 서평, 목차 수집
        for p in ['book_info','book_review','index']:
            if p not in plus_ls:
                plus_dic[p].append(np.NaN) # 수집된 결과 없을시 널 값 부여
                plus_ref_dic[p].append(np.NaN) # 수집된 결과 없을시 널 값 부여
    else:
        code = 'X'
        
    return code'''

"def naver():\n    # 창 전환\n    driver.switch_to.window(driver.window_handles[1]) \n    driver.implicitly_wait(10)\n    try: # 제목 수집 -부제가 없는지 확인먼저 \n        s_tit = driver.find_element(By.CLASS_NAME,f'bookTitle_sub_title__B0uMS').text\n        m_tit = driver.find_element(By.CLASS_NAME,f'bookTitle_book_name__JuBQ2').text\n        if m_tit not in over_ls:\n            over_ls.append(m_tit)\n            main_title.append(m_tit) # 본\n            sub_title.append(s_tit) # 부\n            check = 0 # 계속 진행\n        else:\n            check = 1 # 나감\n            \n            \n        \n    except:\n        m_tit = driver.find_element(By.CLASS_NAME,f'bookTitle_title_area__fspvB').text\n        if m_tit not in over_ls:\n            over_ls.append(m_tit)\n            main_title.append(m_tit) # 본\n            sub_title.append(np.NaN) # 부\n            check = 0\n        else:\n            check = 1\n\n    if check == 0:\n        driver.implicitly_wait(10)\n        try:\n            info_func() # 저

In [9]:
# 단일권의 ISBN 수집 코드
def one_isbn():
    check = []
    driver.find_element(By.XPATH, f'//*[@id="resultList_div"]/form/div[{1}]/div/div/div/a').click()
    driver.implicitly_wait(10)
#     tit = driver.find_element(By.CSS_SELECTOR,'#contents > div > div.resultViewDetail > div.tit').text
#     title_isbn.append(tit.split(']')[1]) # 타이틀 수집, 현재는 제외된 부분
    # 행 수 카운트
    num = driver.find_elements(By.XPATH, '//*[@id="contents"]/div/div[1]/div[2]/div[2]/ul/li')
    for n in range(len(num)):
        a = num[n].text.split('\n')
        if a[0] == 'ISBN':
            check.append('isbn')
            isbn.append(re.sub(r'[^0-9]', '', a[1].split(' ')[0])) # 저자
        elif a[0] in ['세트ISBN','SET ISBN']:
            check.append('set_isbn')
            set_isbn.append(re.sub(r'[^0-9]', '', a[1].split(' ')[0]))
        elif a[0] in ['제본형식','제본형식(종이)']:
            check.append('book_type')
            book_type.append(a[-1])
        elif a[0] == '판사항':
            check.append('ver')
            ver.append(a[-1])
        elif a[0] == '이전판 정보':
            check.append('last_ver')
            last_ver.append(re.sub(r'[^0-9]', '', a[1]))
        elif a[0] == '형태 및 본문언어':
            check.append('pn_isbn')
            pn_isbn.append(np.NaN)
           # pn_isbn.append(a[1]) 세트의 개별권만 수집하도록 변경됨
        elif a[0] == '키워드':
            check.append('keyword')
            keyword.append(a[1]) 
        elif a[0] in ['가격', '가격정보']:
            if re.sub(r'[^0-9]', '', a[1]) != '':
                price.append(re.sub(r'[^0-9]', '', a[1]))
                check.append('price')
            else:
                price.append(np.NaN)
                check.append('price')
        else:
            pass
    for c in ['book_type','ver','last_ver','pn_isbn','keyword', 'set_isbn', 'price']:
        if c not in check:
            isbn_dic[c].append(np.NaN)
        else:
            pass
    if 'isbn' not in check:  # 새로 반영(단일행으로 구성된 세트ISBN정보가 있음)
        isbn.append(set_isbn)

In [10]:
# 세트권의 각 항목별 수집해야하는 내용 모음
def set_code_search(x):
    driver.find_element(By.XPATH, f'//*[@id="resultList_div"]/form/div[{x}]/div/div/div/a').click()
    driver.implicitly_wait(10)
    num = driver.find_elements(By.XPATH, '//*[@id="contents"]/div/div[1]/div[2]/div[2]/ul/li')
    check = []
    to = driver.find_element(By.XPATH, ' //*[@id="contents"]/div/div[1]/div[1]').text
    isbn_title = ' '.join(to.split(']')[1:]).strip()
    for n in range(len(num)):
        a = num[n].text.split('\n')
        if a[0] == 'ISBN':
            check.append('isbn')
            isbn.append(re.sub(r'[^0-9]', '', a[1].split(' ')[0])) 
        elif a[0] in ['세트ISBN','SET ISBN']:
            check.append('set_isbn')
            set_is = re.sub(r'[^0-9]', '', a[1].split(' ')[0])
            set_isbn.append(set_is)
        elif a[0] in ['제본형식','제본형식(종이)'] :
            check.append('book_type')
            book_type.append(a[-1])
        elif a[0] == '판사항':
            check.append('ver')
            ver.append(a[-1])
        elif a[0] == '이전판 정보':
            check.append('last_ver')
            last_ver.append(re.sub(r'[^0-9]', '', a[1]))
        elif a[0] == '형태 및 본문언어':
            check.append('pn_isbn')
            pni = a[1].split('/')[0]
            pn_isbn.append(re.sub(r'[^0-9]', '', pni))
        elif a[0] == '키워드':
            check.append('keyword')
            keyword.append(a[-1])
        elif a[0] in ['가격', '가격정보']:
            if re.sub(r'[^0-9]', '', a[1]) != '':
                price.append(re.sub(r'[^0-9]', '', a[1]))
                check.append('price')
            else:
                price.append(np.NaN)
                check.append('price')

    for c in ['book_type','ver','last_ver','pn_isbn','keyword', 'price']:
        if c not in check:
            isbn_dic[c].append(np.NaN)
    if 'isbn' not in check:
        isbn.append(set_is)
    if 'set_isbn' not in check:
        set_isbn.append(set_is)
    driver.back()
    driver.implicitly_wait(10)
    return isbn_title

# 세트로 이루어진 정보들중 세트와 개별권 구별 함수
def set_code():
    find_set = driver.find_elements(By.CLASS_NAME, 'dot-list')
    last_idx = []
    first_idx = ''
    for i, fs in enumerate(find_set):
        fs = fs.text.split('\n')
        fs = [f.replace(':', '') for f in fs]
        fs = [f.split(' ')[0] for f in fs]
        print(fs)
        if 'ISBN' not in fs:
            first_idx = i+1
            print(first_idx)
        else:
            last_idx.append(i+1)
    return first_idx,last_idx
 
# 위에서 구한 정보를 토대로 세트권 정보 먼저 수집, (네이버 정보와 연결하기 위함)
def set_code_find(x,y):
    print('set')
    ist = set_code_search(x)
    print('set ok')
    time.sleep(3)
    for i in y:
        ist = set_code_search(i)
        main_title.append(ist)
        for c in [book_info, img, wei, size, tot_page, cate, date, publish, author, sub_title,\
                 index, idx_page, book_review, cate_kb, ref_info, ref_review, ref_index]:   # main_title 안에서 개별권 수집
            c.append(np.NaN)
    print('last ok')

In [11]:
# isbn이 단일권인지 세트권인지 판별하는 함수
def search_isbn(code): 
    if code == 'O':  ## naver에서 isbn이 없는 경우가 존재해서 추가함
        for c in [book_type,ver,last_ver,pn_isbn,keyword,set_isbn, price]:
            c.append(np.NaN)
    elif code == 'X':
        pass
    else:
        driver.get(f'https://www.nl.go.kr/seoji/contents/S80100000000.do?schType=simple&schStr={code}')
        driver.implicitly_wait(10)
        num = driver.find_elements(By.CLASS_NAME, 'resultData')
        if len(num) == 1: # 단일 경우 함수
            one_isbn()
            print('단일isbn 완료')
        elif len(num) > 1:
            x, y = set_code()
            set_code_find(x,y)
        else:
            print('등록된 데이터 없음')
            isbn.append(code)
            for c in [book_type,ver,last_ver,pn_isbn,keyword,set_isbn, price]:
                c.append(np.NaN)
    driver.close() # 종료
    driver.implicitly_wait(10)
    driver.switch_to.window(driver.window_handles[0])
    print('isbn수집 완')

In [30]:
# 판매처 링크를 보유한 상품 판별 및 클릭 함수
def find_new():
    try:
#         elements = driver.find_elements(By.CLASS_NAME,f'bookListItem_vendor__d_xBD')
        elements = driver.find_elements(By.PARTIAL_LINK_TEXT,f'판매처')
        print(len(elements),"상품수")
        for i,ele in enumerate(elements):
            print(i, ele.text)
            ele.send_keys(Keys.ENTER)
            driver.implicitly_wait(10)
            print('상품클릭')
            # 네이버 함수
            try:
                code = naver()
                print(code)
            except:
                '''driver.switch_to.window(driver.window_handles[1])
                driver.close()
                print('-')'''
                code ='X'
                driver.switch_to.window(driver.window_handles[1])
            # isbn 함수
            search_isbn(code)
    except:
        print('find예외')
        driver.implicitly_wait(10)

# driver setting

In [29]:
# 첫화면
## 현 주소는 9급 공무원(인기순)
URL = 'https://search.shopping.naver.com/book/search/category?bookTabType=ALL&catId=50005981&pageIndex=1&pageSize=40&sort=REL'
ops = webdriver.ChromeOptions()
ops.add_experimental_option('excludeSwitches', ['enable-logging'])

sv = Service(executable_path= 'C:\Data\chromedriver')
driver = webdriver.Chrome(service=sv, options=ops)

driver.get(url=URL)

# data crawling

In [27]:
driver.switch_to.window(driver.window_handles[0])

In [31]:
tag = driver.find_element(By.CLASS_NAME,'style_footer__DaTQt')
# somthing element까지 스크롤
action = ActionChains(driver)
moment = WebDriverWait(driver,10).until(EC.presence_of_element_located((By.CLASS_NAME, 'style_footer__DaTQt')))

i = 10
while i<=500:
    print(f'{i-1}페이지')
    action.move_to_element(tag).perform()
    driver.implicitly_wait(10) # time.sleep(10)
    find_new()
    moment
    print('for문 페이지 클릭전', i)
    driver.implicitly_wait(5)
    driver.find_element(By.LINK_TEXT,f'{i}').send_keys(Keys.ENTER)
    i += 1
    print('for문 페이지 클릭후')

9페이지
23 상품수
0 도서
판매처 30
상품클릭
exit
X
isbn수집 완
1 도서
판매처 32
상품클릭
exit
X
isbn수집 완
2 도서
판매처 11
상품클릭
isbn수집 완
3 도서
판매처 101
상품클릭
exit
X
isbn수집 완
4 e북
판매처 3
상품클릭
exit
X
isbn수집 완
5 도서
판매처 21
상품클릭
exit
X
isbn수집 완
6 e북
판매처 1
상품클릭
exit
X
isbn수집 완
7 도서
판매처 18
상품클릭
exit
X
isbn수집 완
8 e북
판매처 2
상품클릭
exit
X
isbn수집 완
9 도서
판매처 29
상품클릭
exit
X
isbn수집 완
10 도서
판매처 38
상품클릭
exit
X
isbn수집 완
11 도서
판매처 43
상품클릭
exit
X
isbn수집 완
12 도서
판매처 18
상품클릭
exit
X
isbn수집 완
13 도서
판매처 43
상품클릭
exit
X
isbn수집 완
14 도서
판매처 25
상품클릭
exit
X
isbn수집 완
15 도서
판매처 39
상품클릭
exit
X
isbn수집 완
16 도서
판매처 54
상품클릭
exit
X
isbn수집 완
17 도서
판매처 25
상품클릭
exit
X
isbn수집 완
18 도서
판매처 21
상품클릭
exit
X
isbn수집 완
19 도서
판매처 26
상품클릭
exit
X
isbn수집 완
20 도서
판매처 22
상품클릭
exit
X
isbn수집 완
21 도서
판매처 24
상품클릭
exit
X
isbn수집 완
22 도서
판매처 26
상품클릭
exit
X
isbn수집 완
for문 페이지 클릭전 10
for문 페이지 클릭후
10페이지
20 상품수
0 도서
판매처 30
상품클릭
exit
X
isbn수집 완
1 도서
판매처 40
상품클릭
exit
X
isbn수집 완
2 도서
판매처 27
상품클릭
exit
X
isbn수집 완
3 도서
판매처 35
상품클릭
exit
X
isbn수집 완
4 도서
판매처 41
상품클릭
exit
X
isbn수집 완
5 도서
판매처 39
상품클릭
e

exit
X
isbn수집 완
43 도서
판매처 31
상품클릭
exit
X
isbn수집 완
for문 페이지 클릭전 16
for문 페이지 클릭후
16페이지
35 상품수
0 도서
판매처 25
상품클릭
exit
X
isbn수집 완
1 도서
판매처 27
상품클릭
exit
X
isbn수집 완
2 도서
판매처 39
상품클릭
exit
X
isbn수집 완
3 도서
판매처 195
상품클릭
exit
X
isbn수집 완
4 e북
판매처 3
상품클릭
exit
X
isbn수집 완
5 도서
판매처 24
상품클릭
exit
X
isbn수집 완
6 도서
판매처 99
상품클릭
exit
X
isbn수집 완
7 e북
판매처 3
상품클릭
exit
X
isbn수집 완
8 도서
판매처 147
상품클릭
exit
X
isbn수집 완
9 e북
판매처 3
상품클릭
exit
X
isbn수집 완
10 도서
판매처 35
상품클릭
exit
X
isbn수집 완
11 도서
판매처 20
상품클릭
exit
X
isbn수집 완
12 도서
판매처 45
상품클릭
exit
X
isbn수집 완
13 e북
판매처 3
상품클릭
exit
X
isbn수집 완
14 도서
판매처 52
상품클릭
exit
X
isbn수집 완
15 e북
판매처 3
상품클릭
exit
X
isbn수집 완
16 도서
판매처 24
상품클릭
exit
X
isbn수집 완
17 도서
판매처 10
상품클릭
exit
X
isbn수집 완
18 도서
판매처 28
상품클릭
exit
X
isbn수집 완
19 도서
판매처 44
상품클릭
exit
X
isbn수집 완
20 도서
판매처 36
상품클릭
exit
X
isbn수집 완
21 도서
판매처 58
상품클릭
exit
X
isbn수집 완
22 도서
판매처 39
상품클릭
exit
X
isbn수집 완
23 도서
판매처 37
상품클릭
exit
X
isbn수집 완
24 도서
판매처 2
상품클릭
exit
X
isbn수집 완
25 도서
판매처 31
상품클릭
exit
X
isbn수집 완
26 도서
판매처 38
상품클릭
exit
X
isbn수집 완
27 도

exit
X
isbn수집 완
30 도서
판매처 58
상품클릭
exit
X
isbn수집 완
31 e북
판매처 3
상품클릭
exit
X
isbn수집 완
for문 페이지 클릭전 23
for문 페이지 클릭후
23페이지
39 상품수
0 도서
판매처 18
상품클릭
exit
X
isbn수집 완
1 도서
판매처 22
상품클릭
exit
X
isbn수집 완
2 e북
판매처 4
상품클릭
exit
X
isbn수집 완
3 도서
판매처 17
상품클릭
exit
X
isbn수집 완
4 도서
판매처 18
상품클릭
exit
X
isbn수집 완
5 도서
판매처 15
상품클릭
exit
X
isbn수집 완
6 도서
판매처 19
상품클릭
exit
X
isbn수집 완
7 도서
판매처 26
상품클릭
exit
X
isbn수집 완
8 도서
판매처 30
상품클릭
exit
X
isbn수집 완
9 도서
판매처 12
상품클릭
exit
X
isbn수집 완
10 도서
판매처 10
상품클릭
exit
X
isbn수집 완
11 도서
판매처 13
상품클릭
exit
X
isbn수집 완
12 도서
판매처 14
상품클릭
exit
X
isbn수집 완
13 도서
판매처 24
상품클릭
exit
X
isbn수집 완
14 도서
판매처 12
상품클릭
exit
X
isbn수집 완
15 도서
판매처 16
상품클릭
exit
X
isbn수집 완
16 도서
판매처 12
상품클릭
exit
X
isbn수집 완
17 도서
판매처 13
상품클릭
exit
X
isbn수집 완
18 도서
판매처 11
상품클릭
exit
X
isbn수집 완
19 도서
판매처 15
상품클릭
exit
X
isbn수집 완
20 도서
판매처 11
상품클릭
exit
X
isbn수집 완
21 도서
판매처 16
상품클릭
exit
X
isbn수집 완
22 도서
판매처 9
상품클릭
exit
X
isbn수집 완
23 도서
판매처 8
상품클릭
exit
X
isbn수집 완
24 도서
판매처 10
상품클릭
exit
X
isbn수집 완
25 도서
판매처 23
상품클릭
exit
X
isbn수집 완
26 도

exit
X
isbn수집 완
18 도서
판매처 6
상품클릭
exit
X
isbn수집 완
19 도서
판매처 6
상품클릭
exit
X
isbn수집 완
20 도서
판매처 22
상품클릭
exit
X
isbn수집 완
21 도서
판매처 10
상품클릭
exit
X
isbn수집 완
22 도서
판매처 10
상품클릭
exit
X
isbn수집 완
23 도서
판매처 1
상품클릭
exit
X
isbn수집 완
24 도서
판매처 10
상품클릭
exit
X
isbn수집 완
25 e북
판매처 3
상품클릭
exit
X
isbn수집 완
26 도서
판매처 23
상품클릭
exit
X
isbn수집 완
27 도서
판매처 7
상품클릭
exit
X
isbn수집 완
28 도서
판매처 1
상품클릭
exit
X
isbn수집 완
29 도서
판매처 3
상품클릭
exit
X
isbn수집 완
30 도서
판매처 6
상품클릭
exit
X
isbn수집 완
31 도서
판매처 6
상품클릭
exit
X
isbn수집 완
32 도서
판매처 10
상품클릭
exit
X
isbn수집 완
33 도서
판매처 28
상품클릭
exit
X
isbn수집 완
34 도서
판매처 4
상품클릭
exit
X
isbn수집 완
35 도서
판매처 1
상품클릭
exit
X
isbn수집 완
36 도서
판매처 7
상품클릭
exit
X
isbn수집 완
37 도서
판매처 18
상품클릭
exit
X
isbn수집 완
for문 페이지 클릭전 30
for문 페이지 클릭후
30페이지
19 상품수
0 도서
판매처 47
상품클릭
exit
X
isbn수집 완
find예외
for문 페이지 클릭전 31
for문 페이지 클릭후
31페이지
31 상품수
0 도서
판매처 15
상품클릭
exit
X
isbn수집 완
1 도서
판매처 3
상품클릭
exit
X
isbn수집 완
2 도서
판매처 2
상품클릭
exit
X
isbn수집 완
3 도서
판매처 11
상품클릭
exit
X
isbn수집 완
4 도서
판매처 6
상품클릭
exit
X
isbn수집 완
5 도서
판매처 25
상품클릭
exit
X
isbn수집

exit
X
isbn수집 완
10 도서
판매처 14
상품클릭
exit
X
isbn수집 완
11 도서
판매처 12
상품클릭
exit
X
isbn수집 완
12 도서
판매처 3
상품클릭
exit
X
isbn수집 완
13 도서
판매처 10
상품클릭
exit
X
isbn수집 완
14 도서
판매처 15
상품클릭
exit
X
isbn수집 완
15 도서
판매처 8
상품클릭
exit
X
isbn수집 완
16 e북
판매처 3
상품클릭
exit
X
isbn수집 완
17 도서
판매처 3
상품클릭
exit
X
isbn수집 완
18 도서
판매처 8
상품클릭
exit
X
isbn수집 완
19 도서
판매처 18
상품클릭
exit
X
isbn수집 완
20 도서
판매처 4
상품클릭
exit
X
isbn수집 완
for문 페이지 클릭전 38
for문 페이지 클릭후
38페이지
41 상품수
0 도서
판매처 6
상품클릭
exit
X
isbn수집 완
1 도서
판매처 8
상품클릭
exit
X
isbn수집 완
2 도서
판매처 28
상품클릭
exit
X
isbn수집 완
3 도서
판매처 6
상품클릭
exit
X
isbn수집 완
4 도서
판매처 8
상품클릭
exit
X
isbn수집 완
5 도서
판매처 6
상품클릭
exit
X
isbn수집 완
6 도서
판매처 6
상품클릭
exit
X
isbn수집 완
7 도서
판매처 6
상품클릭
exit
X
isbn수집 완
8 도서
판매처 7
상품클릭
exit
X
isbn수집 완
9 도서
판매처 30
상품클릭
exit
X
isbn수집 완
10 e북
판매처 3
상품클릭
exit
X
isbn수집 완
11 도서
판매처 6
상품클릭
exit
X
isbn수집 완
12 도서
판매처 6
상품클릭
exit
X
isbn수집 완
13 도서
판매처 10
상품클릭
exit
X
isbn수집 완
14 도서
판매처 42
상품클릭
exit
X
isbn수집 완
15 도서
판매처 6
상품클릭
exit
X
isbn수집 완
16 도서
판매처 6
상품클릭
exit
X
isbn수집 완
17 도서
판매처 10
상품클릭
e

exit
X
isbn수집 완
20 도서
판매처 8
상품클릭
exit
X
isbn수집 완
21 도서
판매처 2
상품클릭
exit
X
isbn수집 완
22 도서
판매처 23
상품클릭
exit
X
isbn수집 완
23 도서
판매처 5
상품클릭
exit
X
isbn수집 완
24 도서
판매처 6
상품클릭
exit
X
isbn수집 완
25 도서
판매처 14
상품클릭
exit
X
isbn수집 완
26 도서
판매처 5
상품클릭
exit
X
isbn수집 완
27 도서
판매처 8
상품클릭
exit
X
isbn수집 완
28 도서
판매처 9
상품클릭
exit
X
isbn수집 완
29 도서
판매처 8
상품클릭
exit
X
isbn수집 완
30 도서
판매처 6
상품클릭
exit
X
isbn수집 완
31 도서
판매처 6
상품클릭
exit
X
isbn수집 완
32 도서
판매처 14
상품클릭
exit
X
isbn수집 완
33 도서
판매처 5
상품클릭
exit
X
isbn수집 완
34 도서
판매처 6
상품클릭
exit
X
isbn수집 완
35 도서
판매처 9
상품클릭
exit
X
isbn수집 완
36 도서
판매처 6
상품클릭
exit
X
isbn수집 완
37 도서
판매처 6
상품클릭
exit
X
isbn수집 완
38 도서
판매처 11
상품클릭
exit
X
isbn수집 완
for문 페이지 클릭전 45
for문 페이지 클릭후
45페이지
37 상품수
0 도서
판매처 6
상품클릭
exit
X
isbn수집 완
1 도서
판매처 6
상품클릭
exit
X
isbn수집 완
2 도서
판매처 6
상품클릭
exit
X
isbn수집 완
3 도서
판매처 6
상품클릭
exit
X
isbn수집 완
4 도서
판매처 9
상품클릭
exit
X
isbn수집 완
5 도서
판매처 13
상품클릭
exit
X
isbn수집 완
6 도서
판매처 24
상품클릭
exit
X
isbn수집 완
7 도서
판매처 6
상품클릭
exit
X
isbn수집 완
8 도서
판매처 6
상품클릭
exit
X
isbn수집 완
9 도서
판매처 6
상품클릭
exit


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"link text","selector":"47"}
  (Session info: chrome=114.0.5735.199)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x0112A813+48355]
	(No symbol) [0x010BC4B1]
	(No symbol) [0x00FC5358]
	(No symbol) [0x00FF09A5]
	(No symbol) [0x00FF0B3B]
	(No symbol) [0x0101E232]
	(No symbol) [0x0100A784]
	(No symbol) [0x0101C922]
	(No symbol) [0x0100A536]
	(No symbol) [0x00FE82DC]
	(No symbol) [0x00FE93DD]
	GetHandleVerifier [0x0138AABD+2539405]
	GetHandleVerifier [0x013CA78F+2800735]
	GetHandleVerifier [0x013C456C+2775612]
	GetHandleVerifier [0x011B51E0+616112]
	(No symbol) [0x010C5F8C]
	(No symbol) [0x010C2328]
	(No symbol) [0x010C240B]
	(No symbol) [0x010B4FF7]
	BaseThreadInitThunk [0x77017D59+25]
	RtlInitializeExceptionChain [0x77C9B74B+107]
	RtlClearBits [0x77C9B6CF+191]


In [ ]:
'''for c in [book_type,ver,last_ver,pn_isbn,keyword,set_isbn, price]:
            c.append(np.NaN)'''

In [ ]:
'''for i in [1,2]:
    ist = set_code_search(i)
    main_title.append(ist)
    for c in [book_info, img, wei, size, tot_page, cate, date, publish, author, sub_title,\
            index, idx_page, book_review, cate_kb,ref_info, ref_review, ref_index]:   # main_title 안에서 개별권 수집
        c.append(np.NaN)'''

In [32]:
len(last_ver),len(ref_info),len(book_type),len(img),len(pn_isbn)

(24, 24, 24, 24, 24)

In [33]:
len(tot_page),len(isbn),len(main_title),len(cate),len(price)

(24, 24, 24, 24, 24)

In [ ]:
driver.switch_to.window(driver.window_handles[0])

In [34]:
ref_info = [ri.replace('알라딘에', '알라딘') for ri in ref_info]

AttributeError: 'float' object has no attribute 'replace'

# stored

In [35]:
test = pd.DataFrame({'main_title': main_title, 'sub_title':sub_title,'cate':cate, 'tot_page':tot_page,\
                     'author':author,'publish':publish, 'date':date, 'size':size, 'wei':wei,'img':img,\
                    'book_info':book_info, 'ref_info':ref_info,'book_review':book_review,\
                     'ref_review':ref_review,'index':index,'ref_index':ref_index, \
                     'set_isbn':set_isbn, 'isbn':isbn,'price':price, 'book_type':book_type[-1], 'ver':ver,\
                     'last_ver':last_ver,'pn_isbn':pn_isbn[-1],'keyword':keyword, 'cate_kb':cate_kb})
test.to_excel('C:/police_data/police_book2_07_05.xlsx',encoding = 'utf-8-sig', index=False)

C:\Data\conda\lib\site-packages\pandas\util\_decorators.py:211: FutureWarning: the 'encoding' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'encoding'
  return func(*args, **kwargs)


In [36]:
idx_test = pd.DataFrame({'main_title': main_title,'isbn':isbn,'index': index})
idx_test.to_excel('C:/police_data/police_idx2_07_05.xlsx',encoding = 'utf-8-sig', index=False)

C:\Data\conda\lib\site-packages\pandas\util\_decorators.py:211: FutureWarning: the 'encoding' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'encoding'
  return func(*args, **kwargs)


In [ ]:
# test = pd.DataFrame({'main_title': main_title[:185], 'sub_title':sub_title[:185],'cate':cate[:185], 'tot_page':tot_page[:185],\
#                      'author':author[:185],'publish':publish[:185], 'date':date[:185], 'size':size[:185], 'wei':wei[:185],'img':img[:185],\
#                     'book_info':book_info[:185], 'book_review':book_review[:185],'index':index[:185],'ref_ls':ref_ls[:185], \
#                      'set_isbn':set_isbn[:185], 'isbn':isbn[:185],'price':price[:185], 'book_type':book_type[:185], 'ver':ver[:185],\
#                      'last_ver':last_ver[:185],'pn_isbn':pn_isbn[:185],'keyword':keyword[:185], 'cate_kb':cate_kb[:185]})
# test.to_excel('C:/general_data/general_book_test42_07_03.xlsx',encoding = 'utf-8-sig', index=False)

In [ ]:
# # 목차 테이블
# idx_test = pd.DataFrame({'main_title': main_title[:185],'isbn':isbn,'index': index[:185]})
# idx_test.to_excel('C:/general_data/genera_idx42_07_03.xlsx',encoding = 'utf-8-sig', index=False)